# Stitch JHU data through various schema changes
* Reshape
* See what columns we need to derive

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [2]:
# https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
    # Add some other ones we found applicable
    'US Virgin Islands': 'VI', 
    'United States Virgin Islands': 'VI',
    'Grand Princess': 'Grand Princess',
    'Diamond Princess': 'Diamond Princess', 
    'From Diamond Princess': 'Diamond Princess', 
    'Diamond Princess cruise ship': 'Diamond Princess'
}

# reverse the dict
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

## Pre 2/14

In [3]:
pre214_cases_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_2019-ncov-Confirmed.csv"
pre214_deaths_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_2019-ncov-Deaths.csv"
pre214_recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_2019-ncov-Recovered.csv" 

In [4]:
cases1 = pd.read_csv(pre214_cases_url)
deaths1 = pd.read_csv(pre214_deaths_url)
recovered1 = pd.read_csv(pre214_recovered_url)

Modified helper function, since columns are datetime, will extract date portion.

In [5]:
def parse_columns(df):
    """
    quick helper function to parse columns into values
    uses for pd.melt
    """
    df.columns = df.columns.str.split(' ').str[0]
    columns = list(df.columns)
    id_vars, dates = [], []

    for c in columns:
        if c.endswith("20"):
            dates.append(c)
        else:
            id_vars.append(c)
    return id_vars, dates

# Rename geography columns to be the same as future schemas
def rename_geog_cols(df):
    df.rename(columns = {'Country/Region':'Country_Region', 
                         'Province/State': 'Province_State', 
                         'Long': 'Lon'}, inplace = True)
    return df

In [6]:
# Define some functions we'll use to get totals
# Calculate US State totals
def us_state_totals(df):
    
    state_grouping_cols = ['Country_Region', 'state_abbrev', 'date']
    
    state_totals = df.groupby(state_grouping_cols).agg(
        {'cases':'sum', 'recovered':'sum', 'deaths':'sum'})
    
    state_totals.rename(columns = {'cases': 'state_cases',
                                  'recovered':'state_recovered', 
                                  'deaths': 'state_deaths'}, inplace = True)
    
    df = pd.merge(df, state_totals, on = state_grouping_cols)
    
    return df


# Calculate non-US Province_State totals
def province_totals(df):
    
    province_grouping_cols = ['Country_Region', 'Province_State', 'date']

    province_totals = df.groupby(province_grouping_cols).agg(
        {'cases':'sum', 'recovered':'sum', 'deaths':'sum'})
    
    province_totals.rename(columns = {'cases': 'state_cases',
                                  'recovered':'state_recovered', 
                                  'deaths': 'state_deaths'}, inplace = True)
    
    df = pd.merge(df, province_totals, on = province_grouping_cols) 
    
    return df


# Calculate country totals
def country_totals(df):
    
    country_grouping_cols = ['Country_Region', 'date']
    
    country_totals = df.groupby(country_grouping_cols).agg(
        {'cases':'sum', 'recovered':'sum', 'deaths':'sum'})
    
    country_totals.rename(columns = {'cases': 'country_cases',
                                  'recovered':'country_recovered', 
                                  'deaths': 'country_deaths'}, inplace = True)
    
    df = pd.merge(df, country_totals, on = country_grouping_cols) 
    
    return df

In [7]:
id_vars, dates = parse_columns(cases1)
pre214_df = pd.melt(cases1, id_vars=id_vars, value_vars=dates, value_name="cases", var_name="date",
)

# melt deaths
id_vars, dates = parse_columns(deaths1)
deaths_df = pd.melt(deaths1, id_vars=id_vars, value_vars=dates, value_name="deaths")

# melt recovered
id_vars, dates = parse_columns(recovered1)
recovered_df = pd.melt(
    recovered1, id_vars=id_vars, value_vars=dates, value_name="recovered"
)

# join
pre214_df["deaths"] = deaths_df.deaths
pre214_df["recovered"] = recovered_df.recovered

pre214_df['date'] = pd.to_datetime(pre214_df.date)

part1 = rename_geog_cols(pre214_df)

In [8]:
world1 = part1[part1.Country_Region != 'US'] 
us1 = part1[part1.Country_Region == 'US']  

## Pre 3/23
### This is in 2 groups: 2/15-3/9 and 3/10-3/23, call parts 2 and 3
* part 2 is county level...which need to be summed up to get state totals (subset and keep 2/15 - 3/9)
* part 3 is state level (subset and keep 3/10-3/23)

In [9]:
pre323_cases_url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_19-covid-Confirmed_archived_0325.csv"
pre323_deaths_url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_19-covid-Deaths_archived_0325.csv"
pre323_recovered_url= "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/archived_data/archived_time_series/time_series_19-covid-Recovered_archived_0325.csv"

In [10]:
cases2 = pd.read_csv(pre323_cases_url)
deaths2 = pd.read_csv(pre323_deaths_url)
recovered2 = pd.read_csv(pre323_recovered_url)

In [11]:
id_vars, dates = parse_columns(cases2)
pre323_df = pd.melt(cases2, id_vars=id_vars, value_vars=dates, value_name="cases", var_name="date",
)

# melt deaths
id_vars, dates = parse_columns(deaths2)
deaths_df2 = pd.melt(deaths2, id_vars=id_vars, value_vars=dates, value_name="deaths")

# melt recovered
id_vars, dates = parse_columns(recovered2)
recovered_df2 = pd.melt(
    recovered2, id_vars=id_vars, value_vars=dates, value_name="recovered"
)

# join
pre323_df["deaths"] = deaths_df2.deaths
pre323_df["recovered"] = recovered_df2.recovered

pre323_df['date'] = pd.to_datetime(pre323_df.date)

part2 = rename_geog_cols(pre323_df)

Subset into part2 and part3

In [12]:
start2 = '2/15/2020'
end2 = '3/9/2020'

start3 = '3/10/2020'
end3 = '3/23/2020'

world2 = part2[(part2.Country_Region != 'US') & (part2.date >= start2) & (part2.date <= end2)] 
us2 = part2[(part2.Country_Region == 'US') & (part2.date >= start2) & (part2.date <= end2)]  

world3 = part2[(part2.Country_Region != 'US') & (part2.date >= start3) & (part2.date <= end3)] 
us3 = part2[(part2.Country_Region == 'US') & (part2.date >= start3) & (part2.date <= end3)] 

Clean up each respective part with the right filtering

In [13]:
# us2 has county-level data, but also state and country-level observations. Drop those.
us2 = us2[(us2.Province_State.str.contains(',') == True) | 
              (us2.Province_State.str.contains('Princess') == True)]

us2 = us2[us2.Province_State != 'US']

In [14]:
# us3 has state-level data, but also county and country-level observations. Drop those.
us3 = us3[(us3.Province_State.str.contains(',') == False) | 
              (us3.Province_State.str.contains('Princess') == True)]

us3 = us3[us3.Province_State != 'US']

## Append parts 1-3 together, and do some cleaning

In [15]:
county = us1.append(us2, sort = False)
world = world1.append(world2, sort = False)

In [16]:
# Create state_abbrev column, with special case for the cruise ships
county['state_abbrev'] = county.Province_State.str.split(', ', expand = True)[1]

county['state_abbrev'] = county.apply(lambda row: row.Province_State if row.state_abbrev is None 
                                else row.state_abbrev, axis = 1)

# Create an orig_county columns that stores county-level name. Use to merge later on.
county['orig_county'] = county.Province_State

# Let's remove "county" the name, since sometimes it's Los Angeles County, CA or Los Angeles, CA
county['orig_county'] = county.orig_county.str.replace(' County,', ',')

In [23]:
# Create state_abbrev column
us3['state_abbrev'] = us3.Province_State.map(us_state_abbrev)

Get state and country totals

In [24]:
county = us_state_totals(county)
county = country_totals(county)

world = province_totals(world)
world = country_totals(world)

us3 = us_state_totals(us3)
us3 = country_totals(us3)

In [25]:
# Since we have state/country totals, set the cases, deaths, recovered values to 0 for 3/10-3/23
for col in ['cases', 'deaths', 'recovered']:
    us3[col] = 0

In [26]:
# Append all the US data up to 3/23 together
county = county.append(us3, sort = False)

# Append all the US and world data together up to 3/23 (before most recent massive schema change)
jhu1 = county.append(world, sort = False)

## Get rid of duplicates so far

In [27]:
def some_cleaning(df):
    
    df = df.drop_duplicates(subset = ['Country_Region', 'Lat', 'Lon', 'state_abbrev',
                                            'date', 'cases', 'deaths', 'recovered'])
    
    # If there are still duplicates, it's because JHU sometimes did multiple updates a day
    # This is ok, we'll keep the higher values for cases, deaths, recovered. 
    for col in ['cases', 'deaths', 'recovered']:
        df[col] = df.groupby(['Province_State', 'Country_Region', 
                              'Lat', 'Lon', 'date'])[col].transform('max').fillna(0).astype(int)

    df = df.drop_duplicates(subset = ['Province_State', 'Country_Region',
                                      'Lat', 'Lon', 'date', 'cases', 'deaths', 'recovered'], keep = 'last')

    return df

In [28]:
jhu1 = some_cleaning(jhu1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


## Post 3/23 feature layer

In [29]:
feature_layer_url = "https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/ncov_cases_US/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=OBJECTID%2C+Province_State%2C+Country_Region%2C+Last_Update%2C+Lat%2C+Long_%2C+Confirmed%2C+Recovered%2C+Deaths%2C+Active%2C+Admin2%2C+FIPS%2C+Combined_Key%2C+Incident_Rate%2C+People_Tested&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

cases326 = gpd.read_file(feature_layer_url)

#cases326.to_file(driver = 'GeoJSON', filename = '../data/jhu_feature_layer_3_26_2020.geojson')

In [30]:
cases325 = gpd.read_file('../data/jhu_feature_layer_3_25_2020.geojson')

Need help with Last Update column....it's displaying weird ESRI stuff

In [31]:
cases325['date'] = '3/25/2020'
cases326['date'] = '3/26/2020'

In [52]:
# Append what we have of JHU's new layer so far
post323_df = cases325.append(cases326)

post323_df['date'] = pd.to_datetime(post323_df.date)

In [53]:
def clean_jhu_post323_schema(df):
    
    # Rename columns
    df.rename(columns = {"Long_":"Lon", 
                        "Confirmed":"cases", 
                        "Recovered":"recovered", 
                        "Deaths":"deaths", 
                        "Admin2": "County"} , inplace = True)  
    
    """
    These are the geographic identifiers
    Admin2 = County
    Province_State = US State
    Combined_Key = County, State, Country    
    """
    df['state_abbrev'] = df.Province_State.map(us_state_abbrev)
    df['orig_county'] = df.County + ", " + df.state_abbrev
    
    # Remove the word "County" from orig_county. No difference between Los Angeles County, CA and Los Angeles, CA
    df['orig_county'] = df.orig_county.str.replace(' County,', ',')
    
    # Now change the columns to match with previous schemas
    # Province_State will now display county, state abbrev (Los Angeles, CA)
    df.Province_State = df.orig_county
    
    # Add state and country totals (JHU only collecting US county data now, no more non-US country observations)
    df = us_state_totals(df)
    df = country_totals(df)
    
    # Drop columns
    df = df.drop(columns = ['County', 'Active', 'OBJECTID', 'Last_Update'])
    
    return df

In [54]:
part4 = clean_jhu_post323_schema(post323_df)

In [55]:
part4.columns

Index(['Province_State', 'Country_Region', 'Lat', 'Lon', 'cases', 'recovered',
       'deaths', 'FIPS', 'Combined_Key', 'Incident_Rate', 'People_Tested',
       'geometry', 'date', 'state_abbrev', 'orig_county', 'state_cases',
       'state_recovered', 'state_deaths', 'country_cases', 'country_recovered',
       'country_deaths'],
      dtype='object')

In [56]:
jhu1.columns

Index(['Province_State', 'Country_Region', 'Lat', 'Lon', 'date', 'cases',
       'deaths', 'recovered', 'state_abbrev', 'orig_county', 'state_cases',
       'state_recovered', 'state_deaths', 'country_cases', 'country_recovered',
       'country_deaths'],
      dtype='object')

We have new columns FIPS and Combined_Key in part4. Apply that to jhu1

In [57]:
# Add FIPS and Combined_Key for combined_df3 before appending
fips_key_crosswalk = part4[['orig_county', 'FIPS', 'Combined_Key']]

In [58]:
#fips_key_crosswalk[fips_key_crosswalk.orig_county.str.contains('k, NY')]

In [59]:
jhu2 = pd.merge(jhu1, fips_key_crosswalk, on = 'orig_county', how = 'left')

In [60]:
jhu2[(jhu2.Country_Region =='US')& (jhu2.FIPS.isna() & 
                                    (jhu2.Province_State.str.contains(',')))].Province_State.value_counts()

San Antonio, TX         25
Madison, WI             25
Chicago, IL             25
Tempe, AZ               25
Seattle, WA             25
Boston, MA              25
Jackson County, OR      24
Jefferson Parish, LA    24
Washington, D.C.        24
New York County, NY     24
Name: Province_State, dtype: int64

In [ ]:
jhu2.FIPS = jhu2.loc[jhu2.Province_State == 'Jackson County, OR']

In [62]:
fips_key_crosswalk[fips_key_crosswalk.orig_county.str.contains("Jackson, OR")]

,orig_county,FIPS,Combined_Key
2902,"Jackson, OR",41029,"Jackson, Oregon, US"
6068,"Jackson, OR",41029,"Jackson, Oregon, US"


In [68]:
fips_key_crosswalk[fips_key_crosswalk.orig_county.str.contains("DC")]

,orig_county,FIPS,Combined_Key
3150,"District of Columbia, DC",11001,"District of Columbia,District of Columbia,US"
6315,"District of Columbia, DC",11001,"District of Columbia,District of Columbia,US"


In [65]:
fips_key_crosswalk[fips_key_crosswalk.orig_county.str.contains("Jefferson, LA")]

,orig_county,FIPS,Combined_Key
71,"Jefferson, LA",22051,"Jefferson, Louisiana, US"
3239,"Jefferson, LA",22051,"Jefferson, Louisiana, US"


In [69]:
fips_key_crosswalk[fips_key_crosswalk.orig_county.str.contains("New York")]

,orig_county,FIPS,Combined_Key
1816,"New York City, NY",36061,"New York City, New York, US"
4980,"New York City, NY",36061,"New York City, New York, US"


In [ ]:
manual_fix

In [ ]:
# Make sure FIPS and Combined_Key filled in 
jhu1.groupby()

# Check for duplicates, which come from slightly different spellings of county names

## Append jhu1 and part4

In [ ]:
combined_df3[(combined_df3.Country_Region=='US') & (combined_df3.Province_State == '')]

In [ ]:
pd.merge(combined_df3, fips_key_crosswalk, on = ['orig_county']

In [ ]:
combined_df4 = combined_df3.append(post323_df).sort_values(['Country_Region', 
                                                            'Province_State', 'date'])

In [ ]:
# Add FIPS

combined_df4 = pd.merge(combined_df4.drop(columns = 'FIPS'), county_fips_crosswalk, 
                        on = 'orig_county', how = 'left')

In [ ]:
# Add Combined_Key for US
us2 = combined_df4[combined_df4.Country_Region == "US"]

In [ ]:
combined_key_crosswalk = us2[['FIPS', 'Combined_Key']][us2.FIPS.notna()].drop_duplicates()

In [ ]:
us2 = pd.merge(us2.drop(columns = 'Combined_Key'), combined_key_crosswalk, on = 'FIPS', how = 'left')

In [ ]:
us2[us2.Combined_Key.isna()]

In [ ]:
us2['Combined_Key'] = us2.Province_State.str.split(',').str[0] + ", " + us2.state_abbrev + ", " + us2.Country_Region

In [ ]:
us2.head()

In [ ]:
# Add FIPS, state_abbrev, Combined_Key columns
us['state_abbrev'] = us.Province_State.map(us_state_abbrev)

county_fips_crosswalk = cases3[['orig_county', 'FIPS']].drop_duplicates()

us2 = pd.merge(us, county_fips_crosswalk, on = 'orig_county')

In [ ]:
combined_df2 = us2.append(world)

## Append combined_df2 and cases3

In [ ]:
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry import Point

# Fill in geometry - may not be necessary because we ESRI uses Lat, Lon columns
srid = 4326
df = df.dropna(subset=['Lat', 'Lon'])
df["geometry"] = df.apply(
    lambda x: WKTElement(Point(x.Lon, x.Lat).wkt, srid=srid), axis=1
)

In [ ]:
# Change date column to be datetime....Ian has code for this